# Tracking User Activity

In this project, I've created a service at an ed tech firm that
delivers assessments, and now lots of different customers (e.g., Pearson) want
to publish their assessments on it. I need to prepare for data scientists
who work for these customers to run queries on the data. 

### Tasks

Prepare the infrastructure to land the data in the form and structure it needs
to be to be queried. I need to:

- Publish and consume messages with Kafka
- Use Spark to transform the messages. 
- Use Spark to transform the messages so that I can land them in HDFS

I have included my `docker-compose.yml` used for spinning the pipeline. 

I've also included the history of my console by running
```
history > <user-name>-history.txt
```

I ran Spark by running a Jupyter Notebook against a pyspark kernel.

In order to show the data scientists at these other companies the kinds of data
that they will have access to, I decided on a few basic business questions that
I believed they might need to answer about these data.

### Data

Note on the data: This dataset is a nested JSON file, where I need to unwrap it
carefully to understand what's really being displayed. There are many fields
that were not important for my analysis, so many were left untouched.
The main problem was the multiple questions field. 
Documentation for reading schema implementation in Spark [Here is the documenation from
Apache](https://spark.apache.org/docs/2.3.0/sql-programming-guide.html).


### Business Questions Answered 

1. How many assesstments are in the dataset?
2. How many people took *Learning Git*?
3. How many people got As on their exams?



In [50]:
import json
import pprint
from pyspark.sql import Row

## Linux Commands

### Bring up cluster and ensure that it is running
``` 
docker-compose up -d
docker-compose ps 
docker ps -a
```

### Create the topic
```
docker-compose exec kafka kafka-topics --create --topic assessments --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181
```

### Check the topic

```
docker-compose exec kafka kafka-topics --describe --topic assessments --zookeeper zookeeper:32181 
``` 


### Start the kafka logs 

```
docker-compose logs -f kafka
```

### Put the json objects on the kafka topic
``` 
docker-compose exec mids bash -c "cat /w205/project-2-hgchoi/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t assessments" 
```

### Read the topic with kafkacat to make sure everything is on it

```
docker-compose exec mids bash -c "kafkacat -C -b kafka:29092 -t assessments -o beginning -e" 
```

### Shutdown the cluster
```
docker-compose down
docker-compose up -d
docker-compose ps
docker ps -a
```

## Print json file 

In [51]:
p = pprint.PrettyPrinter(indent=1)

In [52]:
f = open("assessment-attempts-20180128-121051-nested.json","r")

In [53]:
s = f.read()

In [54]:
json_data = json.loads(s)

In [55]:
f.close()

In [56]:
len(json_data)

3280

In [57]:
# this will pretty print the json in alphabetic order which may or may not match the file order
p.pprint(json_data[0])

{'base_exam_id': '37f0a30a-7464-11e6-aa92-a8667f27e5dc',
 'certification': 'false',
 'exam_name': 'Normal Forms and All That Jazz Master Class',
 'keen_created_at': '1516717442.735266',
 'keen_id': '5a6745820eb8ab00016be1f1',
 'keen_timestamp': '1516717442.735266',
 'max_attempts': '1.0',
 'sequences': {'attempt': 1,
               'counts': {'all_correct': False,
                          'correct': 2,
                          'incomplete': 1,
                          'incorrect': 1,
                          'submitted': 4,
                          'total': 4,
                          'unanswered': 0},
               'id': '5b28a462-7a3b-42e0-b508-09f3906d1703',
               'questions': [{'id': '7a2ed6d3-f492-49b3-b8aa-d080a8aad986',
                              'options': [{'at': '2018-01-23T14:23:24.670Z',
                                           'checked': True,
                                           'correct': True,
                                           'id': '

In [58]:
def recursive_walk_json_object(j, level):
    """recursively walk through a json object to explore the structure
       dictionaries will be put in alphabetic order to match the pretty print above"""
    
    level += 1
    
    if type(j) is dict:
        dict_2_list = list(j.keys())
        dict_2_list.sort()
        for k in dict_2_list:
            print("   " * level + "L" + str(level), k)
            recursive_walk_json_object(j[k], level)
    
    elif type(j) is list:
        for (i, l) in enumerate(j):
            print("  " * level + "  [" + str(i) + "]")
            recursive_walk_json_object(l, level)
            
    else:
        print("   " * level + " value:", j)
                    
        
    


In [59]:
recursive_walk_json_object(json_data[0], -1)

L0 base_exam_id
    value: 37f0a30a-7464-11e6-aa92-a8667f27e5dc
L0 certification
    value: false
L0 exam_name
    value: Normal Forms and All That Jazz Master Class
L0 keen_created_at
    value: 1516717442.735266
L0 keen_id
    value: 5a6745820eb8ab00016be1f1
L0 keen_timestamp
    value: 1516717442.735266
L0 max_attempts
    value: 1.0
L0 sequences
   L1 attempt
       value: 1
   L1 counts
      L2 all_correct
          value: False
      L2 correct
          value: 2
      L2 incomplete
          value: 1
      L2 incorrect
          value: 1
      L2 submitted
          value: 4
      L2 total
          value: 4
      L2 unanswered
          value: 0
   L1 id
       value: 5b28a462-7a3b-42e0-b508-09f3906d1703
   L1 questions
      [0]
         L3 id
             value: 7a2ed6d3-f492-49b3-b8aa-d080a8aad986
         L3 options
          [0]
               L5 at
                   value: 2018-01-23T14:23:24.670Z
               L5 checked
                   value: True
               L

### Unroll Assessments Data

In [60]:
raw_assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 


In [61]:
raw_assessments.cache()


DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [105]:
assessments = raw_assessments.select(raw_assessments.value.cast('string'))


In [63]:
extracted_assessments = assessments.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()


In [107]:
extracted_assessments.registerTempTable('assessments')

DataFrame[value: string]


In [65]:
spark.sql("select keen_id from assessments limit 10").show()


+--------------------+
|             keen_id|
+--------------------+
|5a6745820eb8ab000...|
|5a674541ab6b0a000...|
|5a67999d3ed3e3000...|
|5a6799694fc7c7000...|
|5a6791e824fccd000...|
|5a67a0b6852c2a000...|
|5a67b627cc80e6000...|
|5a67ac8cb0a5f4000...|
|5a67a9ba060087000...|
|5a67ac54411aed000...|
+--------------------+



In [66]:
spark.sql("select keen_timestamp, sequences.questions[0].user_incomplete from assessments limit 10").show()


+------------------+-------------------------------------------------------+
|    keen_timestamp|sequences[questions] AS `questions`[0][user_incomplete]|
+------------------+-------------------------------------------------------+
| 1516717442.735266|                                                   true|
| 1516717377.639827|                                                  false|
| 1516738973.653394|                                                  false|
|1516738921.1137421|                                                  false|
| 1516737000.212122|                                                  false|
| 1516740790.309757|                                                  false|
|1516746279.3801291|                                                  false|
| 1516743820.305464|                                                  false|
|  1516743098.56811|                                                  false|
| 1516743764.813107|                                                  false|

### Extract sequence.id by writing a custom lambda transform, creating a separate data frame, registering it as a temp table, and use spark SQL to join it to the outer nesting layer

In [83]:
def my_lambda_sequences_id(x):
    raw_dict = json.loads(x.value)
    my_dict = {"keen_id" : raw_dict["keen_id"], "sequences_id" : raw_dict["sequences"]["id"]}
    return Row(**my_dict)


In [84]:
my_sequences = assessments.rdd.map(my_lambda_sequences_id).toDF()


In [85]:
my_sequences.registerTempTable('sequences')


In [86]:
spark.sql("select sequences_id from sequences limit 10").show()


+--------------------+
|        sequences_id|
+--------------------+
|5b28a462-7a3b-42e...|
|5b28a462-7a3b-42e...|
|b370a3aa-bf9e-4c1...|
|b370a3aa-bf9e-4c1...|
|04a192c1-4f5c-4ac...|
|e7110aed-0d08-4cb...|
|5251db24-2a6e-424...|
|066b5326-e547-4da...|
|8ac691f8-8c1a-403...|
|066b5326-e547-4da...|
+--------------------+



In [87]:
spark.sql("select a.keen_id, a.keen_timestamp, s.sequences_id from assessments a join sequences s on a.keen_id = s.keen_id limit 10").show()


+--------------------+------------------+--------------------+
|             keen_id|    keen_timestamp|        sequences_id|
+--------------------+------------------+--------------------+
|5a17a67efa1257000...|1511499390.3836269|8ac691f8-8c1a-403...|
|5a17a67efa1257000...|1511499390.3836269|8ac691f8-8c1a-403...|
|5a17a67efa1257000...|1511499390.3836269|8ac691f8-8c1a-403...|
|5a17a67efa1257000...|1511499390.3836269|8ac691f8-8c1a-403...|
|5a17a67efa1257000...|1511499390.3836269|8ac691f8-8c1a-403...|
|5a17a67efa1257000...|1511499390.3836269|8ac691f8-8c1a-403...|
|5a17a67efa1257000...|1511499390.3836269|8ac691f8-8c1a-403...|
|5a17a67efa1257000...|1511499390.3836269|8ac691f8-8c1a-403...|
|5a17a67efa1257000...|1511499390.3836269|8ac691f8-8c1a-403...|
|5a26ee9cbf5ce1000...|1512500892.4166169|9bd87823-4508-4e0...|
+--------------------+------------------+--------------------+



### Nested multi-valued as a list: pull out all values from the list by writing a custom labmda transform, creating a another data frame, registering it as a temp table, and joining it to data frames of outer nesting layers

In [88]:
def my_lambda_questions(x):
    raw_dict = json.loads(x.value)
    my_list = []
    my_count = 0
    for l in raw_dict["sequences"]["questions"]:
        my_count += 1
        my_dict = {"keen_id" : raw_dict["keen_id"], "my_count" : my_count, "id" : l["id"]}
        my_list.append(Row(**my_dict))
    return my_list

my_questions = assessments.rdd.flatMap(my_lambda_questions).toDF()

my_questions.registerTempTable('questions')

spark.sql("select id, my_count from questions limit 10").show()

spark.sql("select q.keen_id, a.keen_timestamp, q.id from assessments a join questions q on a.keen_id = q.keen_id limit 10").show()

+--------------------+--------+
|                  id|my_count|
+--------------------+--------+
|7a2ed6d3-f492-49b...|       1|
|bbed4358-999d-446...|       2|
|e6ad8644-96b1-461...|       3|
|95194331-ac43-454...|       4|
|95194331-ac43-454...|       1|
|bbed4358-999d-446...|       2|
|e6ad8644-96b1-461...|       3|
|7a2ed6d3-f492-49b...|       4|
|b9ff2e88-cf9d-4bd...|       1|
|bec23e7b-4870-49f...|       2|
+--------------------+--------+

+--------------------+------------------+--------------------+
|             keen_id|    keen_timestamp|                  id|
+--------------------+------------------+--------------------+
|5a17a67efa1257000...|1511499390.3836269|803fc93f-7eb2-412...|
|5a17a67efa1257000...|1511499390.3836269|f3cb88cc-5b79-41b...|
|5a17a67efa1257000...|1511499390.3836269|32fe7d8d-6d89-4db...|
|5a17a67efa1257000...|1511499390.3836269|5c34cf19-8cfd-4f5...|
|5a17a67efa1257000...|1511499390.3836269|803fc93f-7eb2-412...|
|5a17a67efa1257000...|1511499390.3836269|f3cb88

In [89]:
my_questions = assessments.rdd.flatMap(my_lambda_questions).toDF()



In [90]:
my_questions.registerTempTable('questions')


In [91]:
spark.sql("select id, my_count from questions limit 10").show()


+--------------------+--------+
|                  id|my_count|
+--------------------+--------+
|7a2ed6d3-f492-49b...|       1|
|bbed4358-999d-446...|       2|
|e6ad8644-96b1-461...|       3|
|95194331-ac43-454...|       4|
|95194331-ac43-454...|       1|
|bbed4358-999d-446...|       2|
|e6ad8644-96b1-461...|       3|
|7a2ed6d3-f492-49b...|       4|
|b9ff2e88-cf9d-4bd...|       1|
|bec23e7b-4870-49f...|       2|
+--------------------+--------+



In [92]:
spark.sql("select q.keen_id, a.keen_timestamp, q.id from assessments a join questions q on a.keen_id = q.keen_id limit 10").show()

+--------------------+------------------+--------------------+
|             keen_id|    keen_timestamp|                  id|
+--------------------+------------------+--------------------+
|5a17a67efa1257000...|1511499390.3836269|803fc93f-7eb2-412...|
|5a17a67efa1257000...|1511499390.3836269|f3cb88cc-5b79-41b...|
|5a17a67efa1257000...|1511499390.3836269|32fe7d8d-6d89-4db...|
|5a17a67efa1257000...|1511499390.3836269|5c34cf19-8cfd-4f5...|
|5a17a67efa1257000...|1511499390.3836269|803fc93f-7eb2-412...|
|5a17a67efa1257000...|1511499390.3836269|f3cb88cc-5b79-41b...|
|5a17a67efa1257000...|1511499390.3836269|32fe7d8d-6d89-4db...|
|5a17a67efa1257000...|1511499390.3836269|5c34cf19-8cfd-4f5...|
|5a17a67efa1257000...|1511499390.3836269|803fc93f-7eb2-412...|
|5a17a67efa1257000...|1511499390.3836269|f3cb88cc-5b79-41b...|
+--------------------+------------------+--------------------+



### Handling "holes" in json data

In [93]:
def my_lambda_correct_total(x):
    
    raw_dict = json.loads(x.value)
    my_list = []
    
    if "sequences" in raw_dict:
        
        if "counts" in raw_dict["sequences"]:
            
            if "correct" in raw_dict["sequences"]["counts"] and "total" in raw_dict["sequences"]["counts"]:
                    
                my_dict = {"correct": raw_dict["sequences"]["counts"]["correct"], 
                           "total": raw_dict["sequences"]["counts"]["total"]}
                my_list.append(Row(**my_dict))
    
    return my_list


In [94]:
my_correct_total = assessments.rdd.flatMap(my_lambda_correct_total).toDF()



In [95]:
my_correct_total.registerTempTable('ct')



In [96]:
spark.sql("select * from ct limit 10").show()



+-------+-----+
|correct|total|
+-------+-----+
|      2|    4|
|      1|    4|
|      3|    4|
|      2|    4|
|      3|    4|
|      5|    5|
|      1|    1|
|      5|    5|
|      4|    4|
|      0|    5|
+-------+-----+



In [97]:
spark.sql("select correct / total as score from ct limit 10").show()



+-----+
|score|
+-----+
|  0.5|
| 0.25|
| 0.75|
|  0.5|
| 0.75|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  0.0|
+-----+



In [98]:
spark.sql("select avg(correct / total)*100 as avg_score from ct limit 10").show()



+-----------------+
|        avg_score|
+-----------------+
|62.65699745547132|
+-----------------+



In [99]:
spark.sql("select stddev(correct / total) as standard_deviation from ct limit 10").show()

+------------------+
|standard_deviation|
+------------------+
| 0.310835277631016|
+------------------+



### Write dataframes out to Hadoop HDFS in Parquet format to create a batch and serving layers scale up SQL 

In [103]:
raw_assessments.write.mode('overwrite').parquet("/tmp/raw_assessments")

assessments.write.mode('overwrite').parquet("/tmp/assessments")

extracted_assessments.write.mode('overwrite').parquet("/tmp/extracted_assessments")

my_sequences.write.mode('overwrite').parquet("/tmp/my_sequences")

my_questions.write.mode('overwrite').parquet("/tmp/my_questions")

my_correct_total.write.mode('overwrite').parquet("/tmp/my_correct_total")

### Business Questions and Answers Using Spark SQL against MPP Dataframe/RDD in Memory 

##### 1) How many assessments are in the dataset?

In [78]:
spark.sql("select count(*) as number_of_assessments from assessments").show()

+---------------------+
|number_of_assessments|
+---------------------+
|                 9840|
+---------------------+



##### 2) How many people took the class Learning Git?

In [82]:
spark.sql("select count(*) as students_who_took_learning_git from assessments where exam_name = 'Learning Git'").show()

+------------------------------+
|students_who_took_learning_git|
+------------------------------+
|                          1182|
+------------------------------+



##### 3) How many people got As on their exams?

In [110]:
spark.sql("select count(*) as students_who_got_As from scores where score >= 0.9").show()

+-------------------+
|students_who_got_As|
+-------------------+
|                  4|
+-------------------+



### Write a query, save that query to a new dataframe, register that dataframe as a temp table, then execute a query against the new temp table 

In [79]:
my_scores = spark.sql("select correct / total as score from ct")

In [80]:
my_scores.show()

+------------------+
|             score|
+------------------+
|               0.5|
|              0.25|
|              0.75|
|               0.5|
|              0.75|
|               1.0|
|               1.0|
|               1.0|
|               1.0|
|               0.0|
|              0.75|
|               1.0|
|0.6666666666666666|
|0.6666666666666666|
|               0.8|
|              0.75|
|              0.75|
|               1.0|
|               0.5|
|               1.0|
+------------------+
only showing top 20 rows



In [70]:
my_scores.registerTempTable("scores")

In [71]:
spark.sql("select correct / total as score from ct limit 10").show()

+-----+
|score|
+-----+
|  0.5|
| 0.25|
| 0.75|
|  0.5|
| 0.75|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  0.0|
+-----+



In [72]:
spark.sql("select * from scores").show()

+-----+
|score|
+-----+
|  0.5|
| 0.25|
| 0.75|
|  0.5|
| 0.75|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  0.0|
+-----+



In [73]:
my_scores_gt_70_pct = spark.sql("select * from scores where score > 0.7 ").show()

+-----+
|score|
+-----+
| 0.75|
| 0.75|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
+-----+

